In [1]:
import torch
import torch.nn as nn
import torchvision

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import os

from torch.utils.data import Dataset, DataLoader
# from torch.utils.data.sampler import Sampler
import torch.optim as optim
import sys
sys.path.append('../')
sys.path.append('../../')

from dataset import CocoDetection, train_transforms, val_transforms, test_transforms
from visualize import visualize
# from rcnn_model import fasterrcnn_resnet201_fpn, FastRCNNPredictor
from engine import evaluate
import utils
from models.swin import *

In [2]:
from models.swin_transformer import *
backbone = SwinTransformer()
# backbone

/home/beomgon/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272128894/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [15]:
backbone

SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0): BasicLayer(
      (blocks): ModuleList(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path): Identity()
          (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELU()
            (fc2): Linear(in_features

In [3]:
# layers_to_train = ["layer4", "layer3", "layer2", "layer1", "patch_embed"][:3]
# returned_layers = [1, 2, 3, 4]
# return_layers = {f"layer{k}": str(v) for v, k in enumerate(returned_layers)}
# body = IntermediateLayerGetter(backbone, return_layers=return_layers)
# x = torch.randn(2,3,896,896)
x = torch.randn(2,3,1024,1024)
out = backbone(x)
print(out.keys())


odict_keys(['2', '3', '4'])


In [6]:
# print(out['0'].shape)
# print(out['1'].shape)
print(out['2'].shape)
print(out['3'].shape)
print(out['4'].shape)

torch.Size([2, 192, 128, 128])
torch.Size([2, 384, 64, 64])
torch.Size([2, 768, 32, 32])


In [7]:
from models.detection.retinanet import retinanet_swin_t_fpn
model = retinanet_swin_t_fpn()

trainable_backbone_layers 6


In [8]:
x = torch.randn(2,3,1024,1024)
model.eval()
out = model(x)


In [9]:
out

[{'boxes': tensor([], size=(0, 4), grad_fn=<StackBackward0>),
  'scores': tensor([], grad_fn=<IndexBackward0>),
  'labels': tensor([], dtype=torch.int64)},
 {'boxes': tensor([], size=(0, 4), grad_fn=<StackBackward0>),
  'scores': tensor([], grad_fn=<IndexBackward0>),
  'labels': tensor([], dtype=torch.int64)}]

In [10]:
from models.detection.backbone_utils import *
from ops.feature_pyramid_network import FeaturePyramidNetwork, LastLevelMaxPool
extra_blocks = LastLevelMaxPool()
in_channels_list = [192, 384, 768] 
out_channels = 256
backbone =AttnBackboneWithFPN(backbone, in_channels_list, out_channels, extra_blocks=extra_blocks)

In [11]:
x = torch.randn(2,3,1024,1024)
out = backbone(x)

In [12]:
backbone.fpn

FeaturePyramidNetwork(
  (inner_blocks): ModuleList(
    (0): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1))
    (1): Conv2d(384, 256, kernel_size=(1, 1), stride=(1, 1))
    (2): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1))
  )
  (layer_blocks): ModuleList(
    (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (extra_blocks): LastLevelMaxPool()
)

In [14]:
from models.swin_transformer import *
backbone = SwinTransformer()
x = torch.randn(2,3,1024,1024)
out = backbone(x)
print(out.keys())

odict_keys(['2', '3', '4'])


In [16]:
returned_layers = None
len(returned_layers)

TypeError: object of type 'NoneType' has no len()